# MNIST-Azure
## Test Service
### By: Sebastian Goodfellow

In [1]:
# Configure Notebook
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import 3rd party libraries
import os
import cv2
import sys
import json
import base64
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
from matplotlib.image import imread
from azureml.core import Workspace
from azureml.core.model import Model
from ipywidgets import interact, fixed
from ipywidgets.widgets import IntSlider

# Import local Libraries
sys.path.insert(0, './../')
from mnistazure.config import WORKING_PATH, DATA_PATH

# Get Data

In [2]:
# Image shape
image_shape = (28, 28, 1)

# Get list of validation images
file_names = [file_name for file_name in os.listdir(os.path.join(DATA_PATH, 'images')) 
              if 'val' in file_name]
print(file_names[0:5])

['val_2670.jpg', 'val_4752.jpg', 'val_7445.jpg', 'val_7043.jpg', 'val_3452.jpg']


In [4]:
# Get workspace
workspace = Workspace.get(name='mnist-azure', subscription_id='30284b70-31e1-4b93-b620-26959f80a8f9', 
                          resource_group='spector-ai')

# Get web service
service = workspace.webservices['mnist']
scoring_uri = service.scoring_uri

In [5]:
def plot_prediction(file_name_id, file_names):

    # Get file name
    file_name = file_names[file_name_id]
    
    # Get prediction
    image_array = imread(os.path.join(DATA_PATH, 'images', file_name)).reshape(image_shape)
    test_samples = json.dumps([{'name': 'row1', 'image': encode_image(image_array)}])
    prediction = json.loads(service.run(input_data=test_samples))[0]

    # Plot image
    fig = plt.figure(figsize=(5, 5), facecolor='w')
    fig.subplots_adjust(wspace=0, hspace=1.2)
    ax1 = plt.subplot2grid((1, 1), (0, 0))
    ax1.set_title('Prediction: {}\nScore: {} %'.format(np.argmax(prediction), 
                                                       int(prediction[np.argmax(prediction)] * 100)), 
                  fontsize=16)
    ax1.imshow(image_array[:, :, 0], cmap='gray', vmin=0, vmax=255)
    ax1.axes.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)
    
    plt.show()
    
def encode_image(image):
    """b64 encode RGB numpy array."""
    image_string = cv2.imencode('.jpg', image)[1].tostring()
    return base64.b64encode(image_string).decode("utf-8")

# Launch interactive plotting widget
_ = interact(
    plot_prediction,
    file_name_id=IntSlider(value=0, min=0, max=len(file_names)-1, description='Image ID', disabled=False,),
    file_names=fixed(file_names)
) 

interactive(children=(IntSlider(value=0, description='Image ID', max=9999), Output()), _dom_classes=('widget-i…